In [1]:
getwd()

[1] "/mnt/md0/isshamie/Projects/Mito_Trace/R_scripts/annotations"

## Creates binarized Seurat objects, integrates conditions and annotates genes by nearby peaks

In [2]:
# Input info
#outdir = "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/gff_hg38_1_2/mergedSamples/"
outdir = "/mnt/md0/isshamie/Projects/Mito_Trace/output/CHIP_aggr/data/annotation/gff_A2/mergedSamples"

gff_id = "hg38_1_2"
annotations_indir = "/mnt/md0/isshamie/Projects/Mito_Trace/data/processed/annotation_granges/"


# gene_type="custom" #"UCSC" #or custom
# gene_file = "/data/Mito_Trace/data/processed/genomes/mtMasked/GRCh38_MT_blacklist/genes/genes.gtf"

# Parameters
nTop = 25000
lsi_start_comp = 2
to.filt.cells = "TRUE"
to.qc = FALSE

## QC parameters
min_peak_region_fragments=10
max_peak_region_fragments=15000
min_pct_reads_in_peaks=15
max_blacklist_ratio=0.05
max_nucleosome_signal=4
min_TSS_enrichment=0.2

cores = 24

input_dirs = ""


In [3]:
annotations <- readRDS(file.path(annotations_indir, paste0(gff_id, ".annotationGranges.rds")))

In [4]:
if ((to.filt.cells == "T") | (to.filt.cells == "TRUE")){
    to.filt.cells = TRUE
}else{to.filt.cells = FALSE}


## Add each integrate

In [5]:
library(repr)
options(repr.plot.width=12, repr.plot.height=12)

library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library(dplyr)
library("patchwork")
library(future)
library(Rsamtools)


plan("multiprocess", workers = cores)
options(future.globals.maxSize = 16000 * 1024^2, future.seed=TRUE)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
#plan("multiprocess", workers = workers)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Attaching SeuratObject

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required pa

In [6]:
#allSE <- readRDS(file.path(outdir, paste0("allSamples.rds")))

## Filter cells if parameters there

In [7]:
# filtCells <- function(se, min_peak_region_fragments=10,
#                       max_peak_region_fragments=15000,
#                      min_pct_reads_in_peaks=15,
#                      max_blacklist_ratio=0.05,
#                      max_nucleosome_signal=4,
#                      min_TSS_enrichment=2){
#     print('se before filt')
#     print(se)
#     se <- subset(
#       x = se,
#       subset = peak_region_fragments > min_peak_region_fragments &
#                peak_region_fragments < max_peak_region_fragments &
#                pct_reads_in_peaks > min_pct_reads_in_peaks &
#                blacklist_ratio < max_blacklist_ratio &
#                nucleosome_signal < max_nucleosome_signal  &
#                TSS.enrichment > min_TSS_enrichment
#     )
#     print('se after filt')
#     print(se)
#     return(se)
# }


# allSE



# # Merge:


# # Add sample names to cell prefix here.
# for (i in 1:length(allSE)){
#     print(i)
#     curr.SE <- allSE[[i]]
#     allSE[[i]] <- RenameCells(allSE[[i]], add.cell.id=paste(names(allSE)[[i]],curr.SE$orig.ident[[1]],sep="_"))
#     allSE[[i]]$expID = paste(names(allSE)[[i]],curr.SE$orig.ident[[1]],sep="_")
# }


# length(allSE)

# # merge all datasets, adding a cell ID to make sure cell names are unique
# # combined <- merge(
# #   x = allSE[[1]],
# #   y = unlist(allSE[2:length(allSE)]),
# # )

# if(length(allSE) == 1){
#     combined = allSE[[1]]
# }else{
#     combined <- merge(
#       x = allSE[[1]],
#       y = allSE[[2]])
# }

# if (length(allSE) > 2) {
#     for (i in 3:length(allSE)){
#         combined <- merge(x=combined,
#                           y = allSE[[i]])
#         }
# }

# combined <- FindTopFeatures(combined, min.cutoff = 20)
# combined

# combined[[]]

# combined$expID <- factor(combined$expID)

# VlnPlot(
#   object = combined,
#   features = c('nCount_ATAC', 'peak_region_fragments', 'passed_filters',
#                'duplicate', 'unmapped'),
#   split.by = "expID",
#   pt.size = 0.1,
#   ncol = 3
# )


# # Binarize and run LSI
# combined <- BinarizeCounts(combined)
# combined <- RunTFIDF(combined)
# combined <- RunSVD(combined)
# combined <- RunUMAP(combined, dims = lsi_start_comp:50, reduction = 'lsi')
# DimPlot(combined, group.by = "proj", pt.size = 0.1)

# DimPlot(combined, group.by = "expID", pt.size = 0.1)

# combined[[]]

# pDepthCorr <- DepthCor(combined)
# pDepthCorr

# allSE

# combined[[]]

# allSE[[1]]$exp

# levels(combined$expID)

# allSE <- lapply(levels(combined$expID),  function(x) subset(combined, subset = expID == x))
# allSE


# ### Plot QC first
# to.qc = TRUE

# vPlot <- function(se){
#       vPlot <- VlnPlot(
#       object = se,
#       features = c('pct_reads_in_peaks', 'peak_region_fragments',
#                    'TSS.enrichment', 'blacklist_ratio', 'nucleosome_signal'),
#       pt.size = 0.1,
#       ncol = 5
#     )  
#     vPlot <- vPlot +    # Create grid of plots with title
#              plot_annotation(title = se$expID[[1]]) & 
#              theme(plot.title = element_text(hjust = 0.5, size=15))
#     #print(vPlot)
#     return(vPlot)
# }

# qc <- function(se){
#     # add the gene information to the object
#     Annotation(se) <- annotations
#     gene.activities <- GeneActivity(se)
#     # compute nucleosome signal score per cell
#     se <- NucleosomeSignal(object = se)
    
#     # compute TSS enrichment score per cell
#     se <- TSSEnrichment(object = se, fast = FALSE)

#     # add blacklist ratio and fraction of reads in peaks
#     se$pct_reads_in_peaks <- se$peak_region_fragments / se$passed_filters * 100
#     se$blacklist_ratio <- se$blacklist_region_fragments / se$peak_region_fragments
#     se$high.tss <- ifelse(se$TSS.enrichment > 2, 'High', 'Low')
#     se$nucleosome_group <- ifelse(se$nucleosome_signal > 4, 'NS > 4', 'NS < 4')
#     TSSPlot(se, group.by = 'high.tss') + NoLegend()
#     ggsave(file=file.path(outdir, paste0("QC.TSS", se$expID[[1]], ".png")))
#     FragmentHistogram(object = se, group.by = 'nucleosome_group')
#     ggsave(file=file.path(outdir, paste0("QC.FragHist", se$expID[[1]], ".png")))
#     return(se)
# }

# if (to.qc){
#     #ggsave(file.path(outdir, paste0("QC_02.pdf")))
#     allSE <- lapply(allSE, qc)
#     allSE
#     #ggsave(file.path(outdir,"integrated.batch.png"), dpi=300)
#     }


# ## Filter
# if (to.filt.cells){
#     allSE <- lapply(allSE, filtCells,
#                     min_peak_region_fragments=min_peak_region_fragments,
#                     max_peak_region_fragments=max_peak_region_fragments,
#                     min_pct_reads_in_peaks=min_pct_reads_in_peaks,
#                     max_blacklist_ratio=max_blacklist_ratio,
#                     max_nucleosome_signal=max_nucleosome_signal,
#                     min_TSS_enrichment=min_TSS_enrichment)
# }


# saveRDS(allSE, file.path(outdir, paste0("allSamples.filt.rds")))

In [9]:
combined = readRDS(file.path(outdir, paste0("combined.rds")))
allSE = readRDS(file.path(outdir, paste0("allSamples.filt.rds")))


In [29]:
# May need to filter combined since allSE had QC filter
cells.qc <- unlist(sapply(allSE, function(x) {Cells(x)}))
length(cells.qc)
head(cells.qc)
combined$cell <- row.names(combined[[]])
combined <- subset(combined, subset = cell %in% cells.qc)
combined

[1] 47542

[1] "cd34norm1_Control_AAACGAAAGAGGTCCA-1"
[2] "cd34norm1_Control_AAACGAAAGCGATACG-1"
[3] "cd34norm1_Control_AAACGAAAGTCGTGAG-1"
[4] "cd34norm1_Control_AAACGAACAATAGTGA-1"
[5] "cd34norm1_Control_AAACGAACACAATAAG-1"
[6] "cd34norm1_Control_AAACGAACACTGATAC-1"

In [13]:
if(length(allSE) == 1){
    integrated = allSE[[1]]
}else{
    # find integration anchors
    integration.anchors <- FindIntegrationAnchors(
      object.list = allSE, #c(ext,curr),
      anchor.features = allSE[[1]], #rownames(ext),
      reduction = "rlsi",
      dims = lsi_start_comp:30
    )

    # integrate LSI embeddings
    integrated <- IntegrateEmbeddings(
      anchorset = integration.anchors,
      reductions = combined[["lsi"]],
      new.reduction.name = "integrated_lsi",
      dims.to.integrate = 1:30
    )
}


Computing within dataset neighborhoods

Finding all pairwise anchors



In [ ]:
# integrate LSI embeddings
integrated <- IntegrateEmbeddings(
  anchorset = integration.anchors,
  reductions = combined.filt[["lsi"]],
  new.reduction.name = "integrated_lsi",
  dims.to.integrate = 1:30
)

## Run UMAP and plot

In [ ]:
p1 <- DimPlot(combined, group.by = "expID")

In [ ]:
# create a new UMAP using the integrated embeddings
integrated <- RunUMAP(integrated, reduction = "integrated_lsi", dims = 2:30)
p2 <- DimPlot(integrated, group.by = "expID")
ggsave(file.path(outdir,"integrated.batch.png"), dpi=300)
p2

### Compare merged and integrated

In [ ]:
(p1 + ggtitle("Merged")) | (p2 + ggtitle("Integrated"))
ggsave(file.path(outdir,"integrated.merged.compare.png"), dpi=300)

## Plot sample name

In [ ]:
p.sample <- DimPlot(integrated, group.by = "orig.ident")
(p.sample + ggtitle("Samples after integration"))
ggsave(file.path(outdir,"integrated.sample.png"), dpi=300)

In [ ]:
Fragments(allSE[[1]])

In [ ]:
allSE[[1]]

## Correlation of LSI components and Depth

In [ ]:
pDepthCorr <- DepthCor(integrated, reduction='integrated_lsi')
ggsave(file.path(outdir,"integrated.depthCor.png"), plot=pDepthCorr, dpi=300)

pDepthCorr

In [ ]:
saveRDS(integrated, file.path(outdir, paste0("allSamples.integrated.beforeClust.rds")))

## Cluster and plot

In [ ]:
#integrated <- RunUMAP(object = integrated, reduction = 'integrated_lsi', dims = 2:30)
integrated <- FindNeighbors(object = integrated, reduction = 'integrated_lsi', dims = 2:30)
integrated <- FindClusters(object = integrated, verbose = FALSE, algorithm = 3)

pclust <- DimPlot(object = integrated, label = TRUE) + NoLegend()
ggsave(file.path(outdir, "integrated.lsi.clusters.png"), pclust)
pclust

In [ ]:
saveRDS(integrated, file.path(outdir, paste0("allSamples.integrated.afterClust.rds")))

## Get gene activity results 

In [ ]:
integrated

In [ ]:
integrated[["ATAC"]]

In [ ]:
Fragments(integrated[["ATAC"]])

In [ ]:
Fragments(integrated)

In [ ]:
# add the gene information to the object
DefaultAssay(integrated) <- "ATAC"
Annotation(integrated) <- annotations
gene.activities <- GeneActivity(integrated)


## Remove MT genes

In [ ]:
mt_genes <- annotations[seqnames(annotations) == "chrM"]
dim(mt_genes)


In [ ]:
dim(gene.activities)

In [ ]:
gene.activities <- gene.activities[!(rownames(gene.activities) %in% mt_genes$gene_name),]

In [ ]:
dim(gene.activities)

In [ ]:
# add the gene activity matrix to the Seurat object as a new assay and normalize it
integrated[['RNA']] <- CreateAssayObject(counts = gene.activities)
integrated <- NormalizeData(
  object = integrated,
  assay = 'RNA',
  normalization.method = 'LogNormalize',
  scale.factor = median(integrated$nCount_RNA)
)

In [ ]:
DefaultAssay(integrated) <- 'RNA'

In [ ]:
saveRDS(integrated, file.path(outdir, paste0("allSamples.integrated.rds")))

In [ ]:
Idents(integrated) <- "expID"
VlnPlot(
  object = integrated,
  features = c('nCount_ATAC', 'peak_region_fragments', 'passed_filters',
               'duplicate', 'unmapped'),
  split.by = "expID",
  pt.size = 0.1,
  ncol = 3
)

ggsave(file.path(outdir, paste0("QC_02.png")))

In [ ]:
sessionInfo()

In [ ]:
sessionInfo()

In [ ]:
sessionInfo()